# Exercise solutions

## Exercise 1 - Group 1:
In our analysis we compare the condition 'language' with thecondition  'string'. We do this by utilising a contrast.
1. What do those conditions mean in the context of the experiment?
2. Where in the code is the contrast specified?


In [ ]:
from math import ceil

import matplotlib.pyplot as plt
import numpy as np
from nilearn import plotting # import plotting here

ncols = 3
nrows = ceil(len(models) / ncols)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(8, 4.5))
axes = np.atleast_2d(axes)
model_and_args = zip(models, models_run_imgs, models_events, models_confounds)
for midx, (model, imgs, events, confounds) in enumerate(model_and_args):
    # fit the GLM
    model.fit(imgs, events, confounds)
    # compute the contrast of interest
    zmap = model.compute_contrast("string")  # language - string" or "language" or "string"
    plotting.plot_glass_brain(
        zmap,
        colorbar=False,
        threshold=p001_unc,
        title=f"sub-{model.subject_label}",
        axes=axes[int(midx / ncols), int(midx % ncols)],
        plot_abs=False,
        display_mode="x",
        cmap="bwr",
    )
fig.suptitle("subjects z_map language network (unc p<0.001)")
plotting.show()


3. First, try to change the contrasts so that condition 'language' is compared to the baseline. Then do the same for condition 'string'. Plot both results and compare the plots. What changes can you observe? Interpret them.


In [ ]:
zmap = second_level_model.compute_contrast(
    first_level_contrast="string" # "language - string" or "language" or "string"
)

4. Can you write down design matrixes for each condition? The design matrix for subject one in figure 1 can help you with that.

    If you can, you can also plot those first level analysis design matrixes with NiLearn. The function can be looked up in the documentation.

## Exercise 2 - Group 2:
In figure 2 you can see the z-maps of our participants for an uncorrected p-value of < 0.001.
1. Change the p-value to a different one and re-generate the figure.
2. What changes?
3. What is a good value for the uncorrected p-value and why? Remember the multiple testing problem.

In [ ]:
from scipy.stats import norm

p001_unc = norm.isf(0.001) # the p-value has to be transformed in a z-value before it can be used in other functions

In [ ]:
# If you want to change the p-value only in the first-level analysis
from math import ceil

import matplotlib.pyplot as plt
import numpy as np
from nilearn import plotting # import plotting here

ncols = 3
nrows = ceil(len(models) / ncols)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(8, 4.5))
axes = np.atleast_2d(axes)
model_and_args = zip(models, models_run_imgs, models_events, models_confounds)
for midx, (model, imgs, events, confounds) in enumerate(model_and_args):
    # fit the GLM
    model.fit(imgs, events, confounds)
    # compute the contrast of interest
    zmap = model.compute_contrast("language-string")
    plotting.plot_glass_brain(
        zmap,
        colorbar=False,
        threshold=p001_unc, # change p001_unc to a z-value matching to your p-value, e.g. norm.isf(0.05) for p = 0.05
        title=f"sub-{model.subject_label}",
        axes=axes[int(midx / ncols), int(midx % ncols)],
        plot_abs=False,
        display_mode="x",
        cmap="bwr",
    )
fig.suptitle("subjects z_map language network (unc p<0.001)") # correct the plot label
plotting.show()

## Exercise 3 - Group 3:
As you can see in figure one, we are currently running our analysis on four participants.
1. Can you point out the section of code that is responsible for this sub-selection of participants? How many participants did our dataset originally include?
2. Try to re-run the analysis with a different number of participants and plot the output.

In [ ]:
from nilearn.glm.first_level import first_level_from_bids

task_label = "languagelocalizer"
(
    models,
    models_run_imgs,
    models_events,
    models_confounds,
) = first_level_from_bids(
    data.data_dir,                     
    task_label,
    img_filters=[("desc", "preproc")],
    n_jobs=2, # controls how many tasks are run in parallel (i.e., how many CPU cores are used)
    space_label="",
    sub_labels=["01", "02", "03", "04"],  # comment to run all subjects or add for example "05", "10" to add also subject 5 and 10
)

3. It takes a lot of time to run the analysis on many participants, can the processing time be reduced? Please find the specific parameter and try out different values.

In [ ]:
second_level_model = SecondLevelModel(smoothing_fwhm=8.0, n_jobs=2) # n_jobs controls how many tasks are run in parallel (i.e., how many CPU cores are used)
second_level_model = second_level_model.fit(second_level_input)

## Exercise 4 - Group 4:
What is the p-value we use in our second level analysis? (Hint: look at figure 2)
1. Where in the code is the parameter that sets this value? Can you change it?

In [ ]:
from scipy.stats import norm

p001_unc = norm.isf(0.001) # the p-value (0.001) has to be transformed in a z-value (norm.isf) before it can be used in other functions

2. Try out a few different values and plot them. What changes?

In [ ]:
# If you want to change the p-value only in the second-level analysis
plotting.plot_glass_brain(
    zmap,
    colorbar=True,
    threshold=p001_unc, # change p001_unc to a z-value matching to your p-value, e.g. norm.isf(0.05) for p = 0.05
    title="Group language network (unc p<0.001)", # correct the plot label
    plot_abs=False,
    display_mode="x",
    figure=plt.figure(figsize=(5, 4)),
    cmap="bwr",
)
plotting.show()

3. What is a good p-value for the second level analysis? Keep the multiple testing problem in mind.
4. We use an uncorrected p-value. What methods can we use to correct a p-value? In question 6 we will see how we can implement those correction strategies in NiLearn.

## Exercise 5 - Group 5:
In our second level analysis we can specify a smoothing parameter "smoothing_fwhm".
1. Where can we specifiy this parameter in the code? What is the purpose of this parameter? (Hint: use the documentation)
2. Try a few different values for the smoothing parameter an plot them. What changes in the plots?
3. What are the advantages/disadvantages is using small/large smoothing parameters? Which values would you prefer in which situation?

In [ ]:
second_level_model = SecondLevelModel(smoothing_fwhm=20.0, n_jobs=2) # there is the smoothing parameter
second_level_model = second_level_model.fit(second_level_input)

## Exercise 6 (Advanced, for Python pros) - Group 6:
In our analysis we manually decreased the p-value to 0.001 to adjust the error-rate for the multiple testing problem.
1. Can we use NiLearn to do this automatically? What are the two functions that NiLearn provides for that? (Hint: look at the documentation)
2. What is the best correction method? Extend the code so that this correction method is applied.
3. What effect does the correction have on a p-value of 0.05? Plot two second level analysis output plots side-by-side, one with the uncorrected p-value and the other for the corrected p-value.

In [ ]:
corrected_zmap,threshold_value = nilearn.glm.thresholding.threshold_stats_img(zmap, alpha=0.05, height_control='fpr') # you can change fpr also to 'bonferroni' etc.
plotting.plot_glass_brain(
    corrected_zmap, # the corrected_zmap must be included
    colorbar=True,
    threshold=threshold_value, # the threshold is replaced by the corrected value
    title="Group language network (unc p<0.001)", # correct the plot label
    plot_abs=False,
    display_mode="x",
    figure=plt.figure(figsize=(5, 4)),
    cmap="bwr",
)
plotting.show()